# Initial Setting

In [47]:
import nest_asyncio
nest_asyncio.apply()

import numpy as np
import tensorflow as tf
import random

FRACTION=0.1
BATCH_SIZE = 10 # inf = -1
NUM_EPOCHS = 5 # fixed!
TRAINING_ROUNDS=18

CLIENTS_SHUFFLE_PER_ROUND=False
#CLIENTS_SHUFFLE_PER_ROUND=True 

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
import csv
import time
import sys
import os

class ResultSaver :
  def __init__(self, header) : 
    self.header=header
    now = time.localtime()
    self.result_file_name = "%04d%02d%02d%02d%02d%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec) +'.csv'
    self.f=open('/content/drive/MyDrive/Federated-Learning/result_saver_'+self.result_file_name , 'w', newline='')
    self.wr=csv.writer(self.f)
    self.wr.writerow(header)
    self.content=[]
 
  def put_attr(self, attr) :
    self.content.append(attr)

  def insert_row(self):
    try :
      if(len(self.content)!=len(self.header)):
        raise Exception('Error : not matched header length and row length')
      self.wr.writerow(self.content)
    except Exception as e:
      print(e)
      self.content.clear()
      self.f.close()
      os.rename('/content/drive/MyDrive/Federated-Learning/result_saver_'+self.result_file_name, '/content/drive/MyDrive/Federated-Learning/'+'fail_'+'result_saver_'+self.result_file_name)
      sys.exit(99)
    finally :
      self.content.clear()

  def close_result_saver(self) :
    self.f.close()

#------------------------------------------------------------

rs=ResultSaver(["training_system",
                "compress_ratio",
                "max_round",
                "select_user_ratio",
                "total_user",
                "batch_size",
                "epoch",
                "round",
                "fl_accuracy"
                ])

# rs.put_attr("1")
# rs.put_attr("2")
# rs.insert_row()
# 
# rs.close_result_saver()
# 
# sys.exit()

# AutoEncoder for reducing parameter number

In [50]:
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

def get_compression_ratio(ratio="1:8", ae_input_shape=(25, 32), ae_dense_input_shape=(16, 32)): # It's okay to change shape into numpy array
  seperate_ratio_numerator, seperate_ratio_denominator =  [int(string_el) for string_el in ratio.split(":")]
  ae_end_of_encoder_layer_size=ae_input_shape[0]*(ae_input_shape[1]*(seperate_ratio_numerator/seperate_ratio_denominator))
  ae_dense_end_of_encoder_layer_size=ae_dense_input_shape[0]*(ae_dense_input_shape[1]*(seperate_ratio_numerator/seperate_ratio_denominator))
  return int(ae_end_of_encoder_layer_size), int(ae_dense_end_of_encoder_layer_size)

compression_ratio = "1:8"

ae_end_of_encoder_layer_size, ae_dense_end_of_encoder_layer_size=get_compression_ratio(ratio=compression_ratio)

class Autoencoder(Model):
  def __init__(self):
    super(Autoencoder, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Flatten(), # make origianl 2-dim to 1-dim
      layers.Dense(25*16, activation='relu'),
      layers.Dense(25*8, activation='relu'),
      layers.Dense(ae_end_of_encoder_layer_size, activation='relu'), #### ex) 1:8 -> 25*4
    ])
    self.decoder = tf.keras.Sequential([
      # input dim is 64
      layers.Dense(25*8, activation='relu'),
      layers.Dense(25*16, activation='relu'),
      layers.Dense(25*32, activation='relu'), # It match the encode-flatten()
      layers.Reshape((25, 32)) # restore 1-dim to 2-dim
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder()

class Autoencoder_dense_training(Model):
  def __init__(self):
    super(Autoencoder_dense_training, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Flatten(), # make origianl 2-dim to 1-dim
      layers.Dense(16*16, activation='tanh'),
      layers.Dense(16*8, activation='tanh'),
      layers.Dense(ae_dense_end_of_encoder_layer_size, activation='tanh'), #### ex) 1:8 -> 16*4
    ])
    self.decoder = tf.keras.Sequential([
      # input dim is 64
      layers.Dense(16*8, activation='tanh'),
      layers.Dense(16*16, activation='tanh'),
      layers.Dense(16*32, activation='tanh'), # It match the encode-flatten()
      # layers.Reshape((16, 32)) # restore 1-dim to 2-dim
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder_dense = Autoencoder_dense_training()

autoencoder_dense.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [51]:
autoencoder.load_weights("/content/drive/MyDrive/Federated-Learning/ae_1000_exam_weights.ckpt")

In [52]:
autoencoder_dense.load_weights("/content/drive/MyDrive/Federated-Learning/Model-Parameter/ae_1000_exam_weights.ckpt")

# Make Preprocessed-(I.I.D)Dataset

In [53]:
mnist_train, mnist_test = tf.keras.datasets.mnist.load_data() # This dataset is not "E"mnist. Don't confuse!

raw_dataset_for_iid=list(zip(mnist_train[0].reshape(-1, 28, 28, 1).astype("float32")/255.0, mnist_train[1].astype("float32")))
random.shuffle(raw_dataset_for_iid)

el_size=600
temp_list_for_image=[]
temp_list_for_label=[]
federated_train_data_for_iid=[]
for idx, el in enumerate(raw_dataset_for_iid) :
    temp_list_for_image.append(el[0])
    temp_list_for_label.append(el[1])
    if (idx+1)%(el_size)==0 :
        federated_train_data_for_iid.append((np.array(temp_list_for_image, dtype="float32"), np.array(temp_list_for_label, dtype="float32")))
        temp_list_for_image=[]
        temp_list_for_label=[]
        
federated_train_data = federated_train_data_for_iid

# Make MNIST-CNN 99% model using Keras

In [54]:
keras_model= tf.keras.models.Sequential([
    tf.keras.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, kernel_size=(5, 5), activation="relu", padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    
    tf.keras.layers.Conv2D(64, kernel_size=(5, 5), activation="relu", padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation="softmax"),
])

keras_model.summary()

keras_model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 3136)              0         
_________________________________________________________________
dense_54 (Dense)             (None, 512)               1606144   
_________________________________________________________________
dense_55 (Dense)             (None, 10)              

# Start Training

In [55]:
# def encode_concat_parameter(parameter) :
#     layer1 = parameter[0].reshape(1,25,32)
#     encode_L1 = autoencoder.encoder(layer1).numpy()

#     layer3 = parameter[2].reshape(800,64)
#     layer3_trans = layer3.transpose()
#     layer3_feed_ae = layer3.reshape(64,25,32)
#     encode_L3 = []
#     #for i in range(64):
#     encode_L3 = autoencoder.encoder(layer3_feed_ae).numpy() #.append(autoencoder.encoder(layer3_feed_ae).numpy())

#     temp_list=[encode_L1,     # [layer1] #
#                parameter[1],
#                encode_L3,
#                parameter[3],
#                parameter[4],
#                parameter[5],
#                parameter[6],
#                parameter[7]
#                ] 

#     # for i in range(1,8) :
#     #   temp_list.append(parameter[i])
#     return temp_list

# def decode_seperate_parameter(parameter) :
#     decoded_L1 = autoencoder.decoder(parameter[0]).numpy()  
#     L1_reshape = decoded_L1.reshape(5,5,1,32) #parameter[0].reshape(5,5,1,32) #
#     decoded_L3 = []
#     # for j in range(64):
#     decoded_L3 = autoencoder.decoder(parameter[2]).numpy() #.append(autoencoder.decoder(parameter[2]).numpy())
    
#     decoded_L3_np = np.array(decoded_L3)
#     layer3_reshape_out_ae = decoded_L3_np.reshape(64,800) #parameter[2].reshape(64,800)
#     layer3_before_trans = layer3_reshape_out_ae.transpose()
#     layer3_original = layer3_before_trans.reshape(5,5,32,64)
    
#     temp_list = [L1_reshape,
#                  parameter[1],
#                  layer3_original, #parameter[2],
#                  parameter[3],
#                  parameter[4],
#                  parameter[5],
#                  parameter[6],
#                  parameter[7],
#                  ]

#     # for i in range(1, 8) :
#     #   temp_list.append(parameter[i])
#     return temp_list

In [56]:
def encode_concat_parameter(parameter) :
    # Process of 1st Layer
    layer1 = parameter[0].reshape(1,25,32)
    encode_L1 = autoencoder.encoder(layer1).numpy()
    # Skip 2nd Layer
    # Process of 3rd Layer
    layer3 = parameter[2].reshape(800,64)
    layer3_trans = layer3.transpose()
    layer3_feed_ae = layer3.reshape(64,25,32)
    encode_L3 = []
    #for i in range(64):
    encode_L3 = autoencoder.encoder(layer3_feed_ae).numpy() #.append(autoencoder.encoder(layer3_feed_ae).numpy())

    # Skip 4th Layer
    layer4 = parameter[3]
    # Process of 5th Layer
    layer5 = parameter[4].reshape(3136,512) # 5th layer don't need to be reshape
    encoded_L5 = autoencoder_dense.encoder(layer5).numpy() #.append(autoencoder.encoder(layer3_feed_ae).numpy())   layer5.reshape(3136,16,32)
    # Skip 6th Layer
    # Skip 7th Layer
    # Skip 8th Layer

    temp_list=[encode_L1,     # [layer1] #
               parameter[1],
               encode_L3,
               parameter[3],
               encoded_L5, # parameter[4],
               parameter[5],
               parameter[6],
               parameter[7]
               ] 

    # for i in range(1,8) :
    #   temp_list.append(parameter[i])
    return temp_list

def decode_seperate_parameter(parameter) :
    decoded_L1 = autoencoder.decoder(parameter[0]).numpy()  
    L1_reshape = decoded_L1.reshape(5,5,1,32) #parameter[0].reshape(5,5,1,32) #
    decoded_L3 = []
    # for j in range(64):
    decoded_L3 = autoencoder.decoder(parameter[2]).numpy() #.append(autoencoder.decoder(parameter[2]).numpy())
    decoded_L3_np = np.array(decoded_L3)
    layer3_reshape_out_ae = decoded_L3_np.reshape(64,800) #parameter[2].reshape(64,800)
    layer3_before_trans = layer3_reshape_out_ae.transpose()
    layer3_original = layer3_before_trans.reshape(5,5,32,64)

    # Skip 4th Layer
    layer4 = parameter[3]
    # Process of 5th Layer
    layer5 = autoencoder_dense.decoder(parameter[4]).numpy() # parameter[4] # 5th layer don't need to be reshape
    decoded_L5 = layer5.reshape(3136,512) #autoencoder_dense.decoder(layer5).numpy() #.append(autoencoder.encoder(layer3_feed_ae).numpy())
    # Skip 6th Layer
    # Skip 7th Layer
    # Skip 8th Layer
    
    temp_list = [L1_reshape,
                 parameter[1],
                 layer3_original, #parameter[2],
                 parameter[3],
                 decoded_L5,      #parameter[4],
                 parameter[5],
                 parameter[6],
                 parameter[7],
                 ]

    # for i in range(1, 8) :
    #   temp_list.append(parameter[i])
    return temp_list

In [57]:
fl_system = "mesd"

print("training_system:", fl_system, ", compression ratio:", compression_ratio)
print("max_round:", TRAINING_ROUNDS)

TOTAL_CLIENTS = len(federated_train_data)
SELECTED_CLIENTS = int(TOTAL_CLIENTS*FRACTION)
print("total client :", TOTAL_CLIENTS, ", selected client :", SELECTED_CLIENTS)

# starting to training
selected_clients_list=clients_status_list=np.random.choice(TOTAL_CLIENTS, size=SELECTED_CLIENTS, replace=False) # that is relevant to 4-2 step.

global_parameter=keras_model.get_weights() # actually, It is initial parameter...
encoded_global_parameter=encode_concat_parameter(global_parameter) #### encoder

print("-- parameter shape --")
for layer in global_parameter :
    print(layer.shape)

list_of_local_parameter=[]
list_of_local_dataset_size=[]
list_of_local_accuracy=[]
list_of_local_loss=[]

for round in range(TRAINING_ROUNDS) :
    print("\n▶ Round", round+1, "◀")
    rs.put_attr(fl_system) #### result_saver
    rs.put_attr("1:8") #### result_saver
    rs.put_attr(TRAINING_ROUNDS) #### result_saver
    rs.put_attr(FRACTION) #### result_saver
    rs.put_attr(TOTAL_CLIENTS) #### result_saver
    rs.put_attr(BATCH_SIZE) #### result_saver
    rs.put_attr(NUM_EPOCHS) #### result_saver
    rs.put_attr(round+1) #### result_saver
    
        # check whether to apply shuffle mode per round
    if CLIENTS_SHUFFLE_PER_ROUND == True :
        selected_clients_list = np.random.choice(TOTAL_CLIENTS, size=SELECTED_CLIENTS, replace=False)
    #print("selected clients :", selected_clients_list)

        # recevie Local parameter.
    for client_dataset in selected_clients_list :
        train_images, train_labels=federated_train_data[client_dataset]
        
        decoded_global_parameter = decode_seperate_parameter(encoded_global_parameter)#### decoder

        keras_model.set_weights(decoded_global_parameter)
        
        train_result=keras_model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=0)
            
        local_parameter=keras_model.get_weights()
        encoded_local_parameter= encode_concat_parameter(local_parameter) #### encoder
        list_of_local_parameter.append(encoded_local_parameter)
        list_of_local_dataset_size.append(len(train_images))
        list_of_local_accuracy.append(train_result.history["accuracy"][-1])
        list_of_local_loss.append(train_result.history["loss"][-1])
        
        #print("    clint ID :", client_dataset, "training complete.")
        #print("        accuracy :", train_result.history["accuracy"][-1], "- loss :", train_result.history["loss"][-1])
    
        #4-5. aggregate Local parameters.
    decoded_list_of_loca_parameter=[ decode_seperate_parameter(encoded_local_parameter) for encoded_local_parameter in list_of_local_parameter] #### decoder
    global_parameter = np.mean(decoded_list_of_loca_parameter, axis=0)
    encoded_global_parameter=encode_concat_parameter(global_parameter) #### encoder

    #global_parameter = np.mean(list_of_local_parameter, axis=0)*np.sum)(list_of_local_dataset_size
    #print("global_parameter :",global_parameter)
    current_mean_accuracy = np.mean(np.array(list_of_local_accuracy, dtype="float32"))
    rs.put_attr(current_mean_accuracy) #### result_saver
    current_mean_loss = np.mean(np.array(list_of_local_loss, dtype="float32"))
    print(f"  evaluation mean : accuracy - {current_mean_accuracy}, loss - {current_mean_loss}")   
    
    list_of_local_parameter.clear()
    list_of_local_dataset_size.clear()
    list_of_local_accuracy.clear()
    list_of_local_loss.clear()
    rs.insert_row() #### result_saver
    
print("\n\n▶▶▶ Round is over.")


rs.close_result_saver()

training_system: mesd , compression ratio: 1:8
max_round: 18
total client : 100 , selected client : 10
-- parameter shape --
(5, 5, 1, 32)
(32,)
(5, 5, 32, 64)
(64,)
(3136, 512)
(512,)
(512, 10)
(10,)

▶ Round 1 ◀


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


  evaluation mean : accuracy - 0.2003333568572998, loss - 2.091569185256958

▶ Round 2 ◀
  evaluation mean : accuracy - 0.1211666688323021, loss - 2.295294761657715

▶ Round 3 ◀
  evaluation mean : accuracy - 0.11666665971279144, loss - 2.2951853275299072

▶ Round 4 ◀
  evaluation mean : accuracy - 0.8801666498184204, loss - 0.3885549008846283

▶ Round 5 ◀
  evaluation mean : accuracy - 0.9050000309944153, loss - 0.3044770359992981

▶ Round 6 ◀
  evaluation mean : accuracy - 0.9379999041557312, loss - 0.19136306643486023

▶ Round 7 ◀
  evaluation mean : accuracy - 0.9536666870117188, loss - 0.13894422352313995

▶ Round 8 ◀
  evaluation mean : accuracy - 0.9616665840148926, loss - 0.12049627304077148

▶ Round 9 ◀
  evaluation mean : accuracy - 0.9678333401679993, loss - 0.10664250701665878

▶ Round 10 ◀
  evaluation mean : accuracy - 0.9706667065620422, loss - 0.09201148897409439

▶ Round 11 ◀
  evaluation mean : accuracy - 0.971000075340271, loss - 0.09231776744127274

▶ Round 12 ◀
  e

In [58]:
import sys
sys.exit()

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import pandas as pd
fake_link = "/content/drive/MyDrive/Federated-Learning/fl-model-static-dataset/parameter_set_20210227194250/round_000010/local_parameter_cli_43.xlsx"
local_parameter = pd.read_excel(open(fake_link, 'rb'), index_col=0, sheet_name='dense_1')  

In [ ]:
dfr_sheet2 = pd.read_excel(open(fake_link, 'rb'), index_col=0, sheet_name='conv_layer2')  
layer3 = dfr_sheet2.to_numpy()
print(np.shape(layer3))

In [ ]:
print(layer3.mean())
print(layer3.max()/layer3.mean())
print(layer3.min()/layer3.mean())
print(layer3.max())
print(layer3.min())

In [ ]:
layer5 = local_parameter.to_numpy()
print(np.shape(layer5))
print(layer5[0])


In [ ]:
print(layer5.mean())
print(layer5.max()/layer5.mean())
print(layer5.min()/layer5.mean())
print(layer5.max())
print(layer5.min())

In [ ]:
print(global_parameter[4].mean())
print(global_parameter[4].max()/layer5.mean())
print(global_parameter[4].min()/layer5.mean())
print(global_parameter[4].max())
print(global_parameter[4].min())

In [ ]:
import matplotlib.pyplot as plt
rng = np.random.RandomState(10)  # deterministic random data
a = np.hstack((rng.normal(size=1000),
               rng.normal(loc=5, scale=2, size=1000)))
_ = plt.hist(a, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram of neural network in LeNet-5")
Text(0.5, 1.0, "Histogram of neural network in LeNet-5")
plt.show()

In [ ]:
encoded_L5 = autoencoder_dense.encoder(layer5[0].reshape(1,16,32)).numpy()
decoded_L5 = autoencoder_dense.decoder(encoded_L5).numpy()
print(decoded_L5.reshape(1,512))

In [ ]:
MSE = np.square(np.subtract(layer5[0].reshape(1,16,32),decoded_L5.reshape(1,16,32))).mean() 
print(MSE)

In [ ]:
import matplotlib.pyplot as plt

_ = plt.hist(layer5, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram of neural network in LeNet-5")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

for i in range(3136):
  _ = plt.hist(layer5[i], bins='auto')  # arguments are passed to np.histogram
  plt.title("Histogram of neural network in LeNet-5")
  plt.show()

In [ ]:
import matplotlib.pyplot as plt

_ = plt.hist(global_parameter[4], bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram of neural network in LeNet-5")
plt.show()